# CHAPTER2 データの登録と更新

## 2.7 基本データのバインディング

DOM更新などをバインディングを行う際には使われるデータは`リアクティブデータ`として定義される必要がある。
Vue.jsではこの`リアクティブデータ`が更新された時のみ、DOMの自動更新が行われる。
* リアクティブデータとは
    * Vue.jsが監視しているデータのこと
        * ここで言う監視とは、代入や取得が行われたときフック処理が登録されていることを指す。
    * つまり、フック処理が登録されている`反応できるデータ`のことを`リアクティブデータ`と呼ぶ。

この章では、ディレクティブを用いたデータバインディングの基本的な使い方を学ぶ。


### リアクティブデータの定義

コンポーネントの`data`オプションにリアクティブにするデータを定義することができる。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!' // messageの変化をVue.jsが監視する
    }
});
```
また、`data`で定義された変数は、オプションのブロック外でも変更を検知できる
```js
const state = {
    count: 0;
};
const app = new Vue({
    el: '#app',
    data: {
        state: state
    }
});
state.count = 1; // Vue.jsが監視しているので、DOMが更新される`
```
`data`で定義されるプロパティは後から追加することはできない。
初期値が定まっていない場合でも、後から使う必要がある場合は空データとして定義しておく必要がある。
この辺りは、オブジェクト指向のインスタンス変数のコンストラクタ内での初期化と似た概念。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: null
    }
});
app.message = 'Hello Vue.js!'; // ここで初めてmessageが定義される
```




## 2.8 テキストと属性のデータバインディング

`マスタッシュ`と呼ばれる`{{}}`で囲まれた式をテキストとして表示することができる。
マスタッシュ記法で書かれた式を`マスタッシュタグ`と呼ぶ。このタグは自動的にバインディングされる。
```html
<div id="app">
    <p>{{ message }}</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!'
    }
});
```
バインディングでは、ルートに定義されたプロパティだけでなく、ネストされたオブジェクトも参照可能。
```html
<div id="app">
    <p>{{ list[2] }}</p>
    <p> {{ list[index] }}</p>
    <p>{{ list.length }}</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        list: ['りんご', 'ばなな', 'いちご'],
        index: 1
    }
});
```
実際にレンダリングされると、以下のようになる。
```html
<div id="app">
    <p>いちご</p>
    <p>ばなな</p>
    <p>3</p>
</div>
```

マスタッシュはあくまでJSの式をバインディングできるのであって、文をバインディングすることはできない。
```html
<div id="app">
    <p>{{ message = 'Hello Vue.js!' }}</p> <!-- これはエラーになる -->
</div>
```

### 属性のデータバインディング
マスタッシュはテキストデータ固有の記法のため、属性の指定には使えない。
```html
<div id="app">
    <input type="text" value="{{ message }}"> <!-- これはエラーになる -->
</div>
```
属性へのバインドの場合は、`v-bind`ディレクティブを使う。
```html
<div id="app">
    <input type="text" v-bind:value="message">
</div>
```
`v-bind`ディレクティブは、`:`と省略記法がある。
```html
<div id="app">
    <input type="text" :value="message"> <!-- 省略記法:v-bind:value -->
</div>
```
* `v-bind`の修飾子
    * `v-bind`では以下の修飾子をつけることができる
    
    |修飾子|機能|
    |-|-|
    |`.prop`|属性ではなく、DOMプロパティにバインドする。|
    |`.camel`|キャメルケースに変換する。例:property-name → propertyName|
    |`.sync`|双方向バインディングを行う。|

    * `.prop`の例
    ```html
    <div v-bind:text-content.prop="message"></div>
    <div v-bind:scroll-top.prop="scroll"></div>
    ```
    ```js
    var app = new Vue({
        el: '#app',
        data: {
            message: 'Hello Vue.js!',
            scroll: 0
        }
        mounted: function() {
            this.scroll = 100;
        }
    });
    ```

### クラススタイルとデータバインディング

HTMLのclass属性やstyle属性にデータをバインドするとき、オブジェクトや配列を使うことができる。
その際は、一般のHTML同様、値にしたい文字列をキーに指定する
```html
<div id="app">
    <p v-bind:class="{ `child`: isChild, `is-active`: isActive }">Hello Vue.js!</p>
    <p v-bind:style="{ color: textColor, backgroundColor: bgColor }">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        isChild: true,
        isActive: true,
        textColor: 'red',
        bgColor: 'yellow'
    }
});
```
実際にレンダリングされると、以下のようになる。
```html
<div id="app">
    <p class="child is-active">Hello Vue.js!</p>
    <p style="color: red; background-color: yellow;">Hello Vue.js!</p>
</div>
```

* すでにある値と一緒にしようしたい場合
    * 既存の値と併用した場合、`class`属性と`style`属性では挙動が異なる
    * class属性 : 追加される
    * style属性 : 同じプロパティは上書きされる
    ```html
    <div id="app">
        <p class="pre-class" v-bind:class="{ `is-active`: usActive }">Hello Vue.js!</p>
        <p style="color:black" v-bind:style="{ color: textColor }"></p>
    </div>
    ```
    ```js
    var app = new Vue({
        el: '#app',
        data: {
            isActive: true,
            textColor: 'red'
        }
    });
    ```
    実際にレンダリングされると、以下のようになる。
    ```html
    <div id="app">
        <p class="pre-class is-active">Hello Vue.js!</p>
        <p style="color: red;"></p>
    </div>
    ```

* オブジェクトデータの渡し方
```html
<div id="app">
    <p v-bind:class="classObject">Hello Vue.js!</p>
    <p v-bind:style="styleObject">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        classObject: {
            'child': true,
            'is-active': true
        },
        styleObject: {
            color: 'red',
            backgroundColor: 'yellow'
        }
    }
});
```

### 複数属性のデータバインディング
大量のプロパティを従来の方法でバインディングさせようとすると、テンプレートがすごく長くなってしまう。
```html
<div id="app">
    <p 
    v-bind:id="id" 
    v-bind:class="classObject" 
    v-bind:style="styleObject">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        id: 1,
        classObject: {
            'child': true,
            'is-active': true
        },
        styleObject: {
            color: 'red',
            backgroundColor: 'yellow'
        }
    }
});
```

このような場合、`v-bind`の引数を初裏yくしてオブジェクトを渡すことで、簡潔に記述することができる。
```html
<div id="app">
    <p v-bind="bind-item">Hello Vue.js!</p>
    <!--
        オブジェクトをそのまま渡すことが可能
    -->
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        bindItem: {
            id: 1,
            classObject: {
                'child': true,
                'is-active': true
            },
            styleObject: {
                color: 'red',
                backgroundColor: 'yellow'
            }
        }
    }
});
```
引数を持った`v-bind`と併用することで、特定のプロパティに変更を加えることも可能
```html
<div id="app">
    <img v-bind="item" v-bind:id=" 'thumb-' + item.id ">
</div>
```

### SVGのデータバインディング

Vue.jsはSVGのDOMにも対応しているので、簡単に図形操作ができる。

また、SVG自体をコンポーネント化することも可能。




## 2.9 テンプレートにおける条件分岐

Vue.jsはテンプレートベースでの条件分岐にようる描画操作にも対応している。
`v-if`と`v-show`ディレクティブは、付与したHTMLタグの描画に条件を適用する。

どちらも似たような結果を得るが、DOM自体が消えるのと、`display`プロパティの値をいじっているだけと言う違いがある。
```html
<div id="app">
    <p v-if="isShow">Hello Vue.js!</p>
    <p v-show="isShow">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        isShow: true
    }
});
```
* `isShow`が`true`の場合、以下のように描画される。
```html
<div id="app">
    <p>Hello Vue.js!</p>
    <p>Hello Vue.js!</p>
</div>
```
* `isShow`が`false`の場合、以下のように描画される。
```html
<div id="app">
    <!---->
    <p>Hello Vue.js!</p>
</div>
```
* `v-if`は値がfalseの時指定したDOM自体が消え
* `v-show`は値がfalseの時、`display`プロパティの値を`none`に変更するだけ


### `<template>`によるグループ化

`v-if`や`v-show`は、`<template>`タグで囲むことで、複数の要素をグループ化することができる。
```html
<div id="app">
    <template v-if="isShow">
        <p>Hello Vue.js!</p>
        <p>Hello Vue.js!</p>
    </template>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        isShow: true
    }
});
```
* `isShow`が`true`の場合、以下のように描画される。
```html
<div id="app">
    <p>Hello Vue.js!</p>
    <p>Hello Vue.js!</p>
</div>
```

### `v-else`, `v-else-if`ディレクティブ

`v-if`のある要素の直下に`v-else`, `v-else-if`を記載することで、条件分岐を行うことができる。
```html
<div id="app">
    <p v-if="type === A">Hello Vue.js!</p>
    <p v-else-if="type === B">Good Bye Vue.js!</p>
    <p v-else>Good Bye Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        type: 'A'
    }
});
```
* `type`が`A`の場合、以下のように描画される。
```html
<div id="app">
    <p>Hello Vue.js!</p>
</div>
```

* `v-if`のグループ化で、同じ要素をしようした時は、`key`を指定し、要素を個別化する必要がある。
```html
<div id="app">
    <div v-if="loaded" key="content-visible">
        <p>Hello Vue.js!</p>
        <p>Hello Vue.js!</p>
    </div>
    <div v-else key="content-hidden">
        <p>Now Loading...</p>
    </div>
</div>
<!--
    2つのdivが別要素であることを示す。
-->
```

## 2.10 リストデータの表示と更新

リストデータを定義するとき、以下のように、オブジェクトを内包した配列にすると、仮想DOMによる描画の最適化の恩恵を受けられる。

```js
[
    {id:1, name: 'Taro', age: 35},
    {id:2, name: 'Hanako', age: 29},
    {id:3, name: 'Jiro', age: 18},
]
```

### 要素を繰り返し描画する `v-for`

繰り返し要素を描画するには、`v-for`ディレクティブを使用する。
```html
<div id="app">
    <ul>
        <li v-for="user in users">
            <!--
                pythonのfor文の時と同様な構文
                <各要素の変数名> in 
                <イテレートするオブジェクト>
            -->
            {{ user.name }} ({{ user.age }})
        </li>
    </ul>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        users: [
            {id:1, name: 'Taro', age: 35},
            {id:2, name: 'Hanako', age: 29},
            {id:3, name: 'Jiro', age: 18},
        ]
    }
});
```
この描画結果は、以下のようになる。
```html
<div id="app">
    <ul>
        <li>Taro (35)</li>
        <li>Hanako (29)</li>
        <li>Jiro (18)</li>
    </ul>
</div>
```

* インデックスとオブジェクトの参照
    * 変数部分を`()`で囲むと、インデックスとオブジェクトの両方を取得できる。
    ```html
    <li v-for="(user, index) in users">
        {{ index }}: 
        {{ user.name }} ({{ user.age }})
    </li>
    ```
    * `Object型`の場合は、値、キー、インデックスの順で受け取ることができる。
    ```html
    <li v-for="(value, key, index) in object">
        {{ index }}: {{ key }} = {{ value }}
    </li>
    ```

* `v-for`における`key`の役割
    * `key`は先に示した通り、`同じ要素を再利用する際に要素をVue.js側で個別認識するためのもの`である。
    * `v-for`では、要素の識別と効率的な描画のため、要素に`key`を指定することが強く推奨されている。
    ```html
    <li v-for="user in users" :key="user.id">
        {{ user.name }} ({{ user.age }})
    </li>
    ```
    * 以降取り扱う、`リストレンダリング`や`トランジション`では`key`の指定は必須。
    * `key`の指定がない場合、リストのある要素が削除された時に、描画されているリスト全てが更新されるため、他の更新していない要素まで更新されてしまう、ので、パフォーマンスが悪くなる。

    

### リストの更新
リストやオブジェクトの更新方法も基本的に素のJSと同じ。

ただし、以下の場合はVue側で更新を検知できないので注意。
* `index`数値を用いた配列の更新
    ```js
    this.list[0] = 'new value'; //これは更新されない
    ```
    * `this.$set`を使うことで、Vue側で更新を検知できる。VueのグローバルAPI
    ```js
    this.$set(this.list, 0, 'new value'); //これは更新される
    ```
    * `$set()`はもともとないデータをリアクティブデータにするためにも使われる
    ```js
    //既存のObjeict型のプロパティに新しいプロパティを追加する
    this.$set(this.obj, 'newProp', 'new value'); //{prop: 'value', newProp: 'new value'}
    ```


* 後から追加されたプロパティの更新
    ```js
    new Vue({
        data: {
            obj: {
                prop: 'value'
            }
        }
    });
    this.obj.prop = 'new value'; //これは更新される
    this.obj.newProp = 'new value'; //これは更新されない
    ```
配列ならば`push`や`unshift`を使うことで、Vue側で更新を検知できる。




### 外部データの読み込み

Ajaxで外部データを取り込むことが可能。`axios`を使う。
```js
new Vue({
    el: '#app',
    data: {
        users: []
    },
    created: function() {
        axios.get('https://jsonplaceholder.typicode.com/users')
            .then(function(response) {
                this.users = response.data;
            }.bind(this))
            .catch(function(error) {
                console.log(error);
            });
    }
});
```

## 2.11 DOMの直接参照　`$el`, `$refs`

バインディングを用いることで、要素の描画更新はVueで行えるようになったが、要素の高さなどDOMを介して取得する必要がある場合は、Vueのインスタンスから`$el`を使ってDOMを直接参照する必要がある。

なお、DOMを参照するので`mounted`フック以降でないと、DOMが存在しないため、参照できない。

```js
new Vue({
    el: '#app',
    mounted: function() {
        console.log(this.$el); //Vueインスタンスが描画された要素 <div id="app">...</div>
    }
});
```

root要素以外の要素を参照する場合は、`$refs`を使う。

```html
<div id="app">
    <p ref="myP">Hello</p>
    <!--
        ref属性をつけることでVueインスタンスの$refsオブジェクトにこの要素が追加される
    -->
</div>
```

```js
new Vue({
    el: '#app',
    mounted: function() {
        console.log(this.$refs.myP); //<p ref="myP">Hello</p>
    }
});
```
なおここまでの説明の通り、`$el`や`$refs`は仮想DOMを介さない変更のため、描画の最適化は行われない。
* リアルDOMを使った変更は仮想DOMに影響を与えない。

## 2.12 テンプレート制御ディレクティブ

### `v-pre`ディレクティブ
このディレクティブを付与した内側のHTMLは全てVueのコンパイルから除外される、
* ので、`{{}}`や`v-`ディレクティブが使えなくなる。そのままHTMLとして描画される。

```html
<div id="app">
    <p v-pre>{{ message }}</p>
    <!--
        このp要素はVueのコンパイルから除外される
        マスタッシュやディレクティブがそのまま描画される
        {{ message }}はそのまま描画される
    -->
</div>
```

### `v-once`ディレクティブ

このディレクティブを付与した要素は、初回のみVueでコンパイルされ、その後は静的コンテンツとして扱われる
* 以降はVueが介在しなので、リアクティブとしては扱われない。
```html
<div id="app">
    <p v-once>Hello {{ message }}</p>
    <!--
        このp要素は初回のみVueでコンパイルされ、その後は静的コンテンツとして扱われる
        ※ message = "Welcome"
        レンダリング後
        <p v-once>Hello Welcome</p>
        その後messageを変更しても、描画は変わらない
    -->
</div>
```

### `v-text`ディレクティブ

単一の`{{}}`と同じ役割
```html
<div id="app">
    <p v-text="message"></p>
    <!--
        <p>{{ message }}</p>と同じ
    -->
    <p v-text="msg2">Hello</p>
    <!--
        <p>{{ msg2 }}</p>と同じ.
        レンダリング後はHelloが消え置き換わる
        ただし、msg2が存在しない場合は、Helloが描画される
    -->
```

### `v-html`ディレクティブ

DOM.innerHTMLと同じ役割.
HTMLをそのまま描画させたい場合に使う
```html
<div id="app">
    <p v-html="html"></p>
    <!--
        レンダリング後
        <p><span style="color: red;">Hello</span></p>
    -->
</div>
```

```js
new Vue({
    el: '#app',
    data: {
        html: '<span style="color: red;">Hello</span>'
    }
});
```

### `v-vcloak`ディレクティブ

Vueインスタンスの準備が終わると取り除かれる。初期化前のマスタッシュが表示されるのを防ぐために使う。
```html
<div id="app" v-cloak>
    <p>{{ message }}</p>
    <!-- display:noneのため、Vue初期化完了まで表示されない -->
</div>
```

```css
[v-cloak] {
    display: none;
}
```